# 2018 PGA Tour Strokes Gained Analysis - Performance from different distance ranges

Data is 2018 PGA Tour Shotlink data courtesy of PGA Tour ShotLink. Let's first read the data and modules:

In [69]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/rshot.csv', sep = ';', low_memory=False)


In [70]:
data.columns

Index(['Tour Code', ' Tour Description', ' Year', ' Tourn.#', ' Player #',
       ' Course #', ' Permanent Tournament #', ' Player First Name',
       ' Player Last Name', ' Round', ' Tournament Name', 'Course Name',
       ' Hole', ' Hole Score', ' Par Value', ' Yardage', '  Shot',
       ' Shot Type(S/P/D)', '# of Strokes', ' From Location(Scorer)',
       ' From Location(Enhanced)', ' To Location(Scorer)',
       ' To Location(Enhanced)', ' Distance', ' Distance to Pin',
       ' In the Hole Flag', ' Around the Green Flag', '1st Putt Flag',
       ' Distance to Hole after the Shot', ' Time', ' Lie', ' Elevation',
       ' Slope', ' X Coordinate', ' Y Coordinate', '  Z Coordinate',
       ' Distance from Center', ' Distance from Edge', ' Date', ' Left/Right',
       ' Strokes Gained/Baseline', ' Strokes Gained Category',
       ' Recovery Shot'],
      dtype='object')

A lot of the columns are not needed for this analysis, so we can remove them. We can also change the name of some of them:

In [71]:
#Column renaming and cleanup
data.columns = data.columns.str.replace(' ', '')
data.columns = data.columns.str.replace('#', 'Nr')
data.columns = data.columns.str.replace('.', '')
data['player_name'] = data['PlayerFirstName'] + " " + data['PlayerLastName']
data_cleaned = data.drop(['XCoordinate', 'YCoordinate', 'ZCoordinate', 'TourCode', 
                          'TournNr', 'CourseNr', 'PermanentTournamentNr', 'TourDescription', 
                          'Lie', 'Slope', 'Elevation', '1stPuttFlag', 
                          'Time', 'Date', 'Yardage', 'DistancefromEdge', 'DistancetoHoleaftertheShot', 
                          'DistancefromCenter', 'ToLocation(Scorer)', 'ToLocation(Enhanced)', 'HoleScore', 'ParValue', 'Left/Right',
                          'PlayerFirstName', 'PlayerLastName', 'IntheHoleFlag', 'AroundtheGreenFlag', 'RecoveryShot', 'NrofStrokes'], axis=1)

For this analysis we are only interested in approach shots, which only includes shots in categories Around the Green and Approach the green. We can thus remove the rest. We can also clean up shots which have no distance data, and tournaments which do not have data in the correct format.

In [72]:
#Create dummy categories for strokes gained category
data_cleaned = pd.concat([data_cleaned,pd.get_dummies(data_cleaned['StrokesGainedCategory'] ) ], axis=1 )
#Remove original category
data_cleaned.drop('StrokesGainedCategory', axis=1, inplace=True)
# Remove pair tournament with invalid data
data_cleaned = data_cleaned[data_cleaned['TournamentName'] != 'Zurich Classic of New Orleans']
#Remove match play tournament
data_cleaned = data_cleaned[~data_cleaned['TournamentName'].str.contains('Match Play')]
#Remove shots from rounds without distance data
data_cleaned = data_cleaned[data_cleaned['DistancetoPin'] != 0]
#Remove penalty and drop shots
data_cleaned = data_cleaned[data_cleaned['ShotType(S/P/D)'] == 'S']
data_cleaned.drop('ShotType(S/P/D)', axis=1, inplace=True)
#Remove putting strokes
data_cleaned = data_cleaned[data_cleaned['Putting'] != 1]
#Remove strokes off the tee (excluding tee shots on par 3s and 3rd shots)
data_cleaned = data_cleaned[data_cleaned['Off the Tee'] != 1]
#Remove the stroke categories since we do not need them anymore
data_cleaned.drop(['Putting', 'Around the Green', 'Approach the Green', 'Off the Tee'], axis = 1, inplace=True )
#Convert from inches to yards
data_cleaned['DtP'] = ( data_cleaned['DistancetoPin'] / 36 ).round(0) 
#Create new FromLocation category
data_cleaned['FromLocation'] = np.where( data_cleaned['FromLocation(Scorer)'].isnull(), data_cleaned['FromLocation(Enhanced)'], data_cleaned['FromLocation(Scorer)'])
#Remove the other columns
data_cleaned.drop(['FromLocation(Scorer)', 'FromLocation(Enhanced)'], axis=1, inplace=True)
data_cleaned.head()

,Year,PlayerNr,Round,TournamentName,CourseName,Hole,Shot,Distance,DistancetoPin,StrokesGained/Baseline,player_name,DtP,FromLocation
1,2018,1810,1,Safeway Open,Silverado Resort and Spa North,1,2,4824,4929,0.505,Phil Mickelson,137.0,Primary Rough
4,2018,1810,1,Safeway Open,Silverado Resort and Spa North,2,1,7128,7308,0.158,Phil Mickelson,203.0,Tee Box
8,2018,1810,1,Safeway Open,Silverado Resort and Spa North,3,2,4994,4917,0.143,Phil Mickelson,137.0,Fairway
12,2018,1810,1,Safeway Open,Silverado Resort and Spa North,4,2,4952,4740,0.225,Phil Mickelson,132.0,Fairway Bunker
16,2018,1810,1,Safeway Open,Silverado Resort and Spa North,5,2,7978,7645,0.499,Phil Mickelson,212.0,Primary Rough


To categorize player performance in different categories, we must first define the distance ranges:
 - Define the distance tresholds. The last one will be everything longer than that distance
 - Give them a label
 - Categorize every shot into one of those labels

In [73]:
#Create categories
distances = [0, 50, 100, 140, 175, 200, 230 ]
distance_categories = []

#Place every shot into one category based on distance
for i in range(0, len(distances) - 1):
    cat_name = str(distances[i]) + "-" + str(distances[i+1]) 
    data_cleaned[cat_name] = ( data_cleaned['DtP'] >= distances[i] ) & ( data_cleaned['DtP'] < distances[i+1] )
    distance_categories.append(cat_name)
#Remaining shots (longer than 250)
data_cleaned[ ">=" + str(distances[-1]) ] = data_cleaned['DtP'] >= distances[-1] 
distance_categories.append(">=" + str(distances[-1]))

#Create one column for category instead of multiple binary features
data_cleaned['dist_cat'] = data_cleaned[distance_categories].idxmax(axis=1)
#Drop old features
data_cleaned.drop(distance_categories, axis=1, inplace=True)
#For each tournament and round, take the average strokes gained per category to adjust for difficulty of course
new_baseline =  data_cleaned.groupby(['TournamentName', 'dist_cat', 'Round']).mean()[['StrokesGained/Baseline']].reset_index(level=[0,2])
#Rename column
new_baseline.rename(index=str, columns={'StrokesGained/Baseline': 'new_baseline'}, inplace=True)
#Merge with shot data
data_cleaned_new = data_cleaned.merge(new_baseline, how='left', left_on=['TournamentName', 'Round', 'dist_cat'], right_on=['TournamentName', 'Round', 'dist_cat'])
#Create normalied strokes gained per shot
data_cleaned_new['adj_sg'] = data_cleaned_new['StrokesGained/Baseline'] - data_cleaned_new['new_baseline']


Now that we have the adjusted strokes gained for each shot, we can get the mean for each player and category for the whole season:

In [74]:
#Create new dataframe with each players mean strokes gained per category over the season

player_category_adj_sg = data_cleaned_new.groupby(['PlayerNr', 'player_name', 'dist_cat'])['adj_sg'].describe()
player_category_adj_sg = player_category_adj_sg[['mean', 'count', 'std']].rename(index = str, columns={"mean": 'adj_sg', 'count': 'shot_count'})
player_category_adj_sg['std_err'] = player_category_adj_sg['std'] / np.sqrt(player_category_adj_sg['shot_count'])
player_category_adj_sg['95_conf_upp'] = 1.96*player_category_adj_sg['std_err'] + player_category_adj_sg['adj_sg']
player_category_adj_sg['95_conf_low'] = -1.96*player_category_adj_sg['std_err'] + player_category_adj_sg['adj_sg']
player_category_adj_sg.head()


adj_sg  shot_count       std   std_err  \
PlayerNr player_name   dist_cat                                             
1098     Jay Don Blake 0-50      0.026175        25.0  0.271074  0.054215   
                       100-140   0.011901        10.0  0.430419  0.136110   
                       140-175   0.049126        23.0  0.487558  0.101663   
                       175-200  -0.003291        20.0  0.276450  0.061816   
                       200-230   0.095387         9.0  0.430325  0.143442   

                                 95_conf_upp  95_conf_low  
PlayerNr player_name   dist_cat                            
1098     Jay Don Blake 0-50         0.132436    -0.080086  
                       100-140      0.278678    -0.254875  
                       140-175      0.248386    -0.150133  
                       175-200      0.117868    -0.124451  
                       200-230      0.376533    -0.185758

In [75]:
#Restructure the dataframe
player_category_adj_sg = player_category_adj_sg.unstack(level=-1).swaplevel(0,1,axis=1).sort_index(level=0, axis=1)
player_category_adj_sg.head()

dist_cat                       0-50                                   \
                        95_conf_low 95_conf_upp    adj_sg shot_count   
PlayerNr player_name                                                   
10213    Dicky Pride      -0.031327    0.097048  0.032861      128.0   
10375    Sean McCarty     -0.197637    0.047673 -0.074982       39.0   
10423    Mike Weir        -0.050356    0.230470  0.090057       42.0   
10505    Shane Bertsch    -0.222628    0.119251 -0.051688       25.0   
10585    Scott McCarron   -0.206819   -0.032440 -0.119630       20.0   

dist_cat                                        100-140                        \
                              std   std_err 95_conf_low 95_conf_upp    adj_sg   
PlayerNr player_name                                                            
10213    Dicky Pride     0.370510  0.032749   -0.145402    0.032427 -0.056487   
10375    Sean McCarty    0.390806  0.062579   -0.290302   -0.019091 -0.154696   
10423    Mike Weir       0.464275  0.071639   -0.117270    0.236181  0.059455   
10505    Shane Bertsch   0.436070  0.087214   -0.053809    0.325194  0.135692   
10585    Scott McCarron  0.198940  0.044484   -1.125949    0.172503 -0.476723   

dist_cat                              ...       50-100                       \
                        shot_count    ...       adj_sg shot_count       std   
PlayerNr player_name                  ...                                     
10213    Dicky Pride          75.0    ...    -0.019598       25.0  0.388860   
10375    Sean McCarty         15.0    ...     0.059067       11.0  0.586727   
10423    Mike Weir            26.0    ...    -0.229971        7.0  0.489204   
10505    Shane Bertsch        17.0    ...     0.063407        7.0  0.236437   
10585    Scott McCarron        6.0    ...    -0.119437        6.0  0.179314   

dist_cat                                >=230                        \
                          std_err 95_conf_low 95_conf_upp    adj_sg   
PlayerNr player_name                                                  
10213    Dicky Pride     0.077772   -0.126318    0.067625 -0.029346   
10375    Sean McCarty    0.176905   -0.046756    0.161278  0.057261   
10423    Mike Weir       0.184902   -0.289051    0.178972 -0.055040   
10505    Shane Bertsch   0.089365   -0.324905   -0.020003 -0.172454   
10585    Scott McCarron  0.073204   -0.139536    0.133096 -0.003220   

dist_cat                                                
                        shot_count       std   std_err  
PlayerNr player_name                                    
10213    Dicky Pride          64.0  0.395802  0.049475  
10375    Sean McCarty         13.0  0.191346  0.053070  
10423    Mike Weir            15.0  0.462410  0.119394  
10505    Shane Bertsch         9.0  0.233343  0.077781  
10585    Scott McCarron        9.0  0.208647  0.069549  

[5 rows x 42 columns]

In [76]:

#Players for plot
players = [
          'Tiger Woods', 
          'Rory McIlroy', 
        #  'Jordan Spieth', 
          'Justin Thomas', 
          'Henrik Stenson', 
          #'Brooks Koepka',
          #'Phil Mickelson',
          #'Keegan Bradley',
        #  'Dustin Johnson',
          #'Jason Day'
         # 'Bryson DeChambeau'
          ]

#For sorting the categories
def getSortValue(category):
    return distance_categories.index(category)






In [77]:
#Plotting
import plotly.plotly as py
import plotly
import plotly.graph_objs as go


plots = []
for player in players:
    playerdf = player_category_adj_sg[player_category_adj_sg.index.get_level_values('player_name') == player]
    playerdf = playerdf.transpose().unstack(level=1)
    playerdf.columns =  [ '95_up', '95_low', 'adj_sg', 'shot_count', 'std', 'std_err']
    playerdf['cat_ind'] = pd.Series(playerdf.index.values).apply(getSortValue).values
    playerdf = playerdf.sort_values('cat_ind')
    pplot = go.Scatter (  
        x = playerdf.index.values, 
        y = playerdf['adj_sg'].round(3), 
        name = player,
        text =  "Nr of shots: " + playerdf['shot_count'].astype(int).astype(str),
        error_y=dict(
            type='data',
            symmetric=True,
            array=1.96*playerdf['std_err'],
        )
    )
    plots.append(pplot)


layout = {"title": "Average strokes gained per shot in category"}
py.iplot({"data":plots, "layout": layout } )


## Results
First thing one notices here is that the errors bars representing the 95% confidence interval is very big. This leads us to believe that all of these results should be taken with a grain of salt due to the small sample size, but there are still a few interesting observations to make:
 - Tiger Woods was the best player on tour between 50 and 100 yards last season (see next table).
 - Justin Thomas, who had a great season, was dominating for shorter approach shots but fell off in the longer categories.
 - Henrik Stenson, the Swedish Iceman (I might be biased), is an absolute monster >175 yards while weaker at shorter distances. His legendary 3-wood had another impressive season judging by the >250 metric. 
 - Rory McIlroy who had a relatively weak season was average across the board, perhaps poking hole in the theory that it is his wedge game which is pulling him down.
 
We can also have a look at the best and worst players in each category:

In [78]:
max_player = pd.DataFrame( columns=['cat', 'player_name', 'adj_sg']) 

min_player = pd.DataFrame( columns=['cat', 'player_name', 'adj_sg'])

for cat in distance_categories:
    min_df = player_category_adj_sg[cat][player_category_adj_sg[cat]['shot_count'] > 30]
    min_row = min_df.loc[[min_df['adj_sg'].idxmin(axis=0)]].reset_index()[['player_name', 'adj_sg']]
    min_row['cat'] = cat
    min_player = min_player.append(min_row, ignore_index=True, sort=False)
    
    max_df = player_category_adj_sg[cat][player_category_adj_sg[cat]['shot_count'] > 30]
    max_row = max_df.loc[[max_df['adj_sg'].idxmax(axis=0)]].reset_index()[['player_name', 'adj_sg']]
    max_row['cat'] = cat
    max_player = max_player.append(max_row, ignore_index=True, sort=False)

display(max_player)
display(min_player)

,cat,player_name,adj_sg
0,0-50,Chase Seiffert,0.163480
1,50-100,Tiger Woods,0.127722
2,100-140,John Oda,0.117221
3,140-175,Ross Fisher,0.058953
4,175-200,Andres Romero,0.181636
5,200-230,Thomas Pieters,0.164239
6,>=230,Henrik Stenson,0.144942


,cat,player_name,adj_sg
0,0-50,Michael Block,-0.202849
1,50-100,Sangmoon Bae,-0.156242
2,100-140,Chris Couch,-0.108338
3,140-175,Brendon Todd,-0.148595
4,175-200,Steven Bowditch,-0.203186
5,200-230,Smylie Kaufman,-0.116193
6,>=230,Kris Blanks,-0.177867


In [79]:
player_category_adj_sg.to_csv('data/player_category_sg_2018')